<a href="https://colab.research.google.com/github/gustavsma/MD/blob/main/DETReg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
import os
from google.colab import drive

drive.mount('/content/gdrive',force_remount=True)

In [ ]:
# %cd /content/gdrive/MyDrive
# !git clone https://github.com/amirbar/DETReg.git

In [ ]:
#!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torch==1.9.0 torchvision==0.10.0 torchaudio==0.9.0
!pip uninstall transformers
!pip install transformers==4.8.0

%cd /content/gdrive/MyDrive/DETReg
!pip install -r requirements.txt

In [ ]:
%cd /content/gdrive/MyDrive/DETReg/models/ops
!sh make.sh

In [ ]:
# %cd /content/gdrive/MyDrive/DETReg/
# !ln -s /content/gdrive/MyDrive/data/ data

In [ ]:
!chmod 755 -R /content/gdrive/MyDrive/DETReg/

# Pretrain

In [ ]:
%cd /content/gdrive/MyDrive/DETReg/
!GPUS_PER_NODE=1 tools/run_dist_launch.sh 1 configs/pre_train.sh --output_dir exps/pretrain --batch_size 2 --epochs 100 --pretrain exps/DETReg_top30_in/checkpoint_coco.pth

# Finetune

### SSL

In [ ]:
%cd /content/gdrive/MyDrive/DETReg/
!GPUS_PER_NODE=1 tools/run_dist_launch.sh 1 configs/fine_tune.sh --pretrain exps/pretrain/checkpoint0079.pth --output_dir exps/weeds_ssl --batch_size 2

### SL

In [ ]:
# %cd /content/gdrive/MyDrive/DETReg/exps/DETReg_top30_in
# !wget https://github.com/amirbar/DETReg/releases/download/1.0.0/checkpoint_coco.pth

In [ ]:
%cd /content/gdrive/MyDrive/DETReg/
!GPUS_PER_NODE=1 tools/run_dist_launch.sh 1 configs/fine_tune.sh --pretrain exps/DETReg_top30_in/checkpoint_coco.pth --output_dir exps/weeds_sl --batch_size 2

# Eval

In [ ]:
%cd /content/gdrive/MyDrive/DETReg/

!configs/fine_tune.sh --resume exps/weeds_ssl/checkpoint0494.pth --eval
#!configs/fine_tune.sh --resume exps/weeds_sl/checkpoint0499.pth --eval